# Problem 1

In [3]:
install.packages(c("boot","msm","dplyr")) #We start by installing and loading necessary packages
library(boot)
library(msm)
library(dplyr)

Installing packages into 'C:/Users/Honza/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'dplyr' is in use and will not be installed"

package 'boot' successfully unpacked and MD5 sums checked
package 'msm' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Honza\AppData\Local\Temp\RtmpqeyR0Q\downloaded_packages


Warning message:
"package 'boot' was built under R version 3.6.3"Warning message:
"package 'msm' was built under R version 3.6.3"
Attaching package: 'msm'

The following object is masked from 'package:boot':

    cav



In [4]:
library(boot)
data(city)
data <- city
data
#data <- data(city) #we load the data
#data #let's take a look, dataset is so small that we can view it as a whole

u,x
138,143
93,104
61,69
179,260
48,75
37,63
29,50
23,48
30,111
2,50


## a) Draw the dependence between $u_i$ and $x_i$

## b)

In [10]:
R = mean(data$x)/mean(data$u)
R
sample_n(data, nrow(data), replace = TRUE)

[1] 1.520312

,u,x
2...1,93,104
5,48,75
7,29,50
10,2,50
8...5,23,48
4,179,260
2...7,93,104
8...8,23,48
2...9,93,104
6,37,63


# c)

In [35]:
# brute force bootstraping
Boot    = 99999
x.boot = rep(0, Boot)
u.boot = rep(0, Boot)

set.seed(320)
for(i in 1:Boot) {
  sample_data <- sample_n(data, nrow(data), replace = TRUE)
  x.boot[i] <- mean(sample_data$x)
  u.boot[i] <- mean(sample_data$u)
}

In [ ]:
# brute force bootstraping
Boot    = 99999
x.boot = rep(0, Boot)
u.boot = rep(0, Boot)

set.seed(3)
for(i in 1:Boot) {
  x.sample <- sample(data$x, nrow(data), replace = TRUE)
  x.boot[i] <- mean(x.sample)
  u.sample <- sample(data$u, nrow(data), replace = TRUE)
  u.boot[i] <- mean(u.sample)
}

In [36]:
g <- x.boot/u.boot
R_bf = mean(g) #bf stands for brute force
sprintf("Bootstrap estimate of R: %.4f", R_bf)
length(u.boot)

[1] "Bootstrap estimate of R: 1.5597"

[1] 99999

In [13]:
r

[1] 1.424779 1.763583 1.467012 1.538750 1.318408 1.403994 1.658206 1.507087
   [9] 1.486076 1.661386 1.874396 1.809935 1.276065 1.570288 1.818182 1.307578
  [17] 1.363846 1.176991 2.313480 1.688372 1.780702 1.407634 1.505470 1.895616
  [25] 1.965770 1.890585 1.410714 1.376136 1.583882 1.722864 1.338442 1.661232
  [33] 1.491339 1.878205 1.465181 1.706897 1.530245 2.259868 1.631696 1.582734
  [41] 1.662088 1.589641 1.912921 1.373970 2.513109 1.536082 1.646489 1.346154
  [49] 1.490196 1.267038 1.509908 1.495885 1.261084 1.526012 1.374663 1.325601
  [57] 1.644643 1.607988 1.542744 1.450867 1.527778 1.200908 1.489676 1.546025
  [65] 1.479137 1.819372 1.357447 1.868787 1.508701 1.499184 1.351220 2.593750
  [73] 1.668954 1.441624 1.182754 1.559055 1.248889 1.662896 1.463964 1.266458
  [81] 1.392357 1.870370 1.363303 1.423402 1.563091 1.375522 2.422383 1.728175
  [89] 1.472062 1.292582 1.290375 1.488462 1.346218 1.527886 1.777336 1.656085
  [97] 1.473761 1.466855 1.759036 1.821687 1.711152 1.596396 1.591696 1.890244
 [105] 1.351438 1.450450 1.338065 1.689587 1.825858 1.507200 1.623188 1.640118
 [113] 1.198559 1.715931 1.736842 1.656101 1.482955 1.505034 1.236641 1.447288
 [121] 1.449206 1.854839 1.458824 1.266667 1.216142 1.760504 1.551786 1.490872
 [129] 1.601340 1.669691 1.664384 1.575621 1.638507 1.392683 1.606719 1.523723
 [137] 1.260695 1.395100 1.154426 1.611785 1.514658 1.625680 1.459514 1.216851
 [145] 1.502899 1.645374 1.390997 1.347255 1.550544 1.298876 1.445479 1.606618
 [153] 1.453159 2.719178 1.337405 1.716279 1.805907 1.733781 1.879925 1.708126
 [161] 1.268897 1.403141 1.732517 1.735441 1.578526 1.704545 1.306723 1.411640
 [169] 1.546970 1.330435 1.409922 1.436131 1.341407 1.542636 1.412727 1.335113
 [177] 1.821883 1.339623 1.431159 1.651709 1.818874 1.499329 1.322459 1.669922
 [185] 1.578035 1.547893 1.929293 1.340000 1.898844 1.511396 1.781124 1.709068
 [193] 1.689256 1.478856 1.575758 2.333333 1.790816 1.625418 1.588721 1.369826
 [201] 1.645078 1.398667 1.519753 1.272509 1.517500 1.444248 1.545977 2.103333
 [209] 1.369880 1.637594 1.544318 1.793496 1.476415 1.366630 1.491368 1.604538
 [217] 1.577875 1.368175 1.244876 1.705882 1.318584 1.893039 1.671280 1.576305
 [225] 1.752336 1.455285 1.502521 1.599715 2.129545 1.540107 1.496109 1.513333
 [233] 1.483703 1.430233 1.360324 2.000000 1.470707 1.603093 1.571212 1.367133
 [241] 1.870748 1.341772 1.828025 1.711712 1.512890 1.819149 1.504909 1.683453
 [249] 1.431452 1.684818 1.501319 1.629630 1.426230 1.402280 1.525481 1.395143
 [257] 1.510204 1.784133 1.337696 1.646417 1.934831 1.388829 1.762646 1.615730
 [265] 1.599644 1.409051 1.582888 1.434954 1.310811 1.439297 1.389769 1.540816
 [273] 1.385841 1.631673 1.560440 1.478022 1.758294 1.945750 1.583480 1.582343
 [281] 1.437282 1.502970 1.462079 1.462274 1.783595 1.489149 1.564516 2.286195
 [289] 1.718699 1.387481 1.202543 1.546875 1.461686 1.553571 1.526241 1.486819
 [297] 1.538462 1.576336 1.662356 1.593789 1.712519 1.634740 1.306195 1.735450
 [305] 1.722338 1.924757 1.425051 1.586538 1.357735 1.706186 1.521605 1.227902
 [313] 1.360825 1.524316 1.552593 1.325887 1.451014 1.484429 1.364544 1.458807
 [321] 1.447980 1.529801 1.565000 1.419540 1.487267 1.472393 1.267606 1.450450
 [329] 1.848485 1.476874 1.317460 1.396907 1.400000 1.478261 2.178571 1.361290
 [337] 1.428105 1.558659 1.345382 1.452804 1.438871 1.626788 1.388740 1.465164
 [345] 1.591168 1.581982 1.278515 1.469613 1.695481 1.678426 1.506369 1.461417
 [353] 1.543590 1.596429 2.282116 1.477011 1.577938 1.892857 1.691517 1.539281
 [361] 1.625578 1.892617 1.280597 2.120321 1.203039 1.804481 1.594595 1.542039
 [369] 1.848000 1.857143 1.428344 1.533639 2.091954 1.833010 1.682051 1.404580
 [377] 1.420775 1.443787 1.699074 1.392966 1.313380 1.379630 1.575365 1.642857
 [385] 1.225490 2.806950 1.354103 2.085714 1.349229 1.411420 1.493955 1.666667
 [393] 2.040512 1.584615 1.270955 1.497253 1.502249 1.549234 1.869494 1.358283
 [401] 1.752475 1.615101 1.493411 1.918182 1.491254 1

# Problem 3

In [ ]:
install.packages(c("readr","tseries","gmm")) #necessary packages
library(readr) #so that we can load the data
library(tseries)
library(gmm)

In [ ]:
data3 <- read_csv("hw2_data.csv")
head(data3)

In [ ]:
install.packages("rlang")